# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_df = pd.read_csv("/Users/victoriasandoval/python-api-challenge/WeatherPy/cities.csv")
city_df


,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,yeppoon,-23.1333,150.7333,65.98,94,100,7.36,AU,1652024137
1,1,biak,-0.9131,122.8766,77.77,86,100,7.18,ID,1652024138
2,2,hobart,-42.8794,147.3294,44.06,88,21,9.22,AU,1652024138
3,3,rikitea,-23.1203,-134.9692,76.51,80,42,21.25,PF,1652024139
4,4,angoche,-16.2325,39.9086,75.34,76,33,7.92,MZ,1652024140
...,...,...,...,...,...,...,...,...,...,...
557,557,yining,29.0361,114.5636,67.03,98,72,0.76,CN,1652024435
558,558,aquiraz,-3.9014,-38.3911,84.29,85,100,8.50,BR,1652024436
559,559,fallon,39.4735,-118.7774,51.67,35,100,12.66,US,1652024436
560,560,maguling,6.0950,124.4033,80.64,85,100,2.84,PH,1652024437


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

In [15]:
locations = city_df[["Lat", "Lng"]].astype(float)
weights = city_df["Humidity"].astype(float)

In [16]:
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

In [17]:
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
ideal_cities = city_df.loc[(city_df["Temp"] > 70) & 
            (city_df["Temp"] < 80) &
            (city_df["Wind"] < 10) & 
            (city_df["Cloudiness"] == 0)].dropna()

ideal_cities


,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
140,140,saint george,37.1041,-113.5841,73.80,15,0,3.44,US,1652023999
179,179,aksu,41.1231,80.2644,79.20,21,0,4.83,CN,1652024229
239,239,cianorte,-23.6633,-52.6050,71.01,57,0,6.40,BR,1652024264
316,316,posadas,-27.3671,-55.8961,73.38,60,0,6.91,AR,1652024306
495,495,elk city,35.4120,-99.4043,79.48,53,0,0.00,US,1652024403
501,501,beira,-19.8436,34.8389,77.09,83,0,5.75,MZ,1652024406


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = ideal_cities.copy()
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
140,140,saint george,37.1041,-113.5841,73.80,15,0,3.44,US,1652023999
179,179,aksu,41.1231,80.2644,79.20,21,0,4.83,CN,1652024229
239,239,cianorte,-23.6633,-52.6050,71.01,57,0,6.40,BR,1652024264
316,316,posadas,-27.3671,-55.8961,73.38,60,0,6.91,AR,1652024306
495,495,elk city,35.4120,-99.4043,79.48,53,0,0.00,US,1652024403


In [37]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
140,140,saint george,37.1041,-113.5841,73.80,15,0,3.44,US,1652023999,
179,179,aksu,41.1231,80.2644,79.20,21,0,4.83,CN,1652024229,
239,239,cianorte,-23.6633,-52.6050,71.01,57,0,6.40,BR,1652024264,
316,316,posadas,-27.3671,-55.8961,73.38,60,0,6.91,AR,1652024306,
495,495,elk city,35.4120,-99.4043,79.48,53,0,0.00,US,1652024403,


In [43]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

In [54]:
for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Long = row["Lng"]

    params["location"] = f"{Lat},{Long}"

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(url, params = params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("not found, skip")

not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip


In [55]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
140,140,saint george,37.1041,-113.5841,73.80,15,0,3.44,US,1652023999,
179,179,aksu,41.1231,80.2644,79.20,21,0,4.83,CN,1652024229,
239,239,cianorte,-23.6633,-52.6050,71.01,57,0,6.40,BR,1652024264,
316,316,posadas,-27.3671,-55.8961,73.38,60,0,6.91,AR,1652024306,
495,495,elk city,35.4120,-99.4043,79.48,53,0,0.00,US,1652024403,
501,501,beira,-19.8436,34.8389,77.09,83,0,5.75,MZ,1652024406,


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lng"]]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))